<a href="https://colab.research.google.com/github/fabriziobasso/Colab_backup/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%capture
# Connect to Colab:#
from google.colab import drive
import os
drive.mount('/content/drive')

In [9]:
folder_script = models_folders = "/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose"
os.chdir(folder_script)

In [10]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [11]:
warnings.filterwarnings('ignore')
#plt.style.use('fivethirtyeight')

# Setting rc parameters in seaborn for plots and graphs-
# Reference - https://matplotlib.org/stable/tutorials/introductory/customizing.html:-
# To alter this, refer to matplotlib.rcParams.keys()

sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.5,
         'grid.alpha'           :0.5,
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : 'black',
         'axes.titlesize'       : 12,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,
        });

sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5})
# Set Style
mpl.rcParams['figure.dpi'] = 120;

# Making sklearn pipeline outputs as dataframe:-
pd.set_option('display.max_columns', 100);
pd.set_option('display.max_rows', 50);

sns.despine(left=True, bottom=True, top=False, right=False)

mpl.rcParams['axes.spines.left'] = True
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.bottom'] = True

<Figure size 768x576 with 0 Axes>

In [12]:
!apt install graphviz
import os
os.environ["PATH"] += os.pathsep + '/usr/bin/dot'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [13]:
ext_data=True

if ext_data==False:
  df_train=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/train_cluster.csv", index_col=0)
  df_test=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/test_cluster.csv", index_col=0)

  df_train_scaled=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_train_scaled.csv", index_col=0)
  df_test.shape

if ext_data==True:
  df_train = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/train_cluster_enc_final.csv", index_col=0)
  df_test = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/test_cluster_enc_final.csv", index_col=0)

  df_train_scaled=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_train_scaled.csv", index_col=0)
  print(df_test.shape,df_train.shape)

(3644, 669) (177024, 670)


In [16]:
target = 'bg+1:00'

trgt = df_train[target]
df_train.drop(columns=[target], inplace=True)

In [29]:
pr_0 = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_conv_v4_selection.csv", index_col=0)
pr_1 = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_conv_v0_average.csv", index_col=0)
pr_2 = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_encoder_v6.csv", index_col=0)
pr_3 = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_conv_v2_average.csv", index_col=0)
pr_4 = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_encoder_v5.csv", index_col=0)


total = pd.concat([pr_0, pr_1, pr_2, pr_3, pr_4], axis=1)

In [30]:
total["average"] = total.mean(axis=1)
total.sample(10)

,bg+1:00,bg+1:00,bg+1:00,bg+1:00,bg+1:00,average
id,,,,,,
p18_21,9.794079,9.497472,9.868514,10.055276,9.421381,9.727344
p04_24849,9.223553,9.230664,9.560540,9.444493,9.235913,9.339033
p12_25502,4.061851,4.440859,4.569306,151.614544,6.695523,34.276417
p19_78,6.349909,6.488942,6.799061,6.343665,6.914008,6.579117
p05_8349,7.692085,7.625792,7.772311,7.513992,7.531467,7.627129
p04_24708,6.976246,6.670115,7.045301,7.018576,6.543663,6.850780
p12_25414,8.468141,8.161466,8.167438,8.486875,8.163143,8.289413
p24_213,13.802224,13.668700,13.720251,13.487644,13.650408,13.665845
p01_8577,7.874329,8.025698,8.572906,8.058537,9.267486,8.359791


#### **STORE RESULTS**

In [35]:
sub = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/sample_submission.csv")
sub

,id,bg+1:00
0,p01_8459,9.6
1,p01_8460,4.6
2,p01_8461,8.0
3,p01_8462,9.9
4,p01_8463,5.3
...,...,...
3639,p24_256,6.3
3640,p24_257,10.4
3641,p24_258,6.9
3642,p24_259,8.7


In [36]:
sub.sample()

,id,bg+1:00
1357,p10_25572,7.4


In [37]:
sub["bg+1:00"] = total["average"].values
sub.to_csv(f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/submissions/Submission_val_conv_v4_selection.csv", index=False)
print(sub.isna().sum())

id         0
bg+1:00    0
dtype: int64
